<a href="https://colab.research.google.com/github/nneko1231/REAL_Video_Enhancer/blob/main/REAL_Video_Enhancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Drive Mount

In [ ]:
# will mount drive if MOUNT_DRIVE is True
import os
import subprocess
import shutil
from google.colab import drive, output

# Add a parameter to control mounting Google Drive
MOUNT_DRIVE = True  #@param {type:"boolean"}

# mount drive if MOUNT_DRIVE is True
if MOUNT_DRIVE:
    drive.mount('/content/drive')

# 2.Settings

In [ ]:
# Input and Output file paths
INPUT_FILE = "input.mp4"  #@param {type:"string"}
OUTPUT_FILE = "output.mkv"  #@param {type:"string"}
KILL_RUNTIME_AFTER_RENDER = True  #@param {type:"boolean"}

#@markdown <hr>

#@markdown ##Interpolate Settings
INTERPOLATE = True  #@param {type:"boolean"}
INTERPOLATE_FACTOR = 2  # @param {type:"number"}
SCENE_DETECT_METHOD = "pyscenedetect"  #@param ["pyscenedetect", "none"]
SCENE_DETECT_SENSATIVITY = 5  #@param {type:"slider", "min":0, "max":9.9, "step":0.1}

"""
#@markdown Interpolate Models:
#@markdown - rife4.6.pkl : Speed
#@markdown - rife4.7.pkl : Smoothness
#@markdown - rife4.18.pkl : IRL
#@markdown - rife4.22.pkl : Animation
#@markdown - rife4.25.pkl : General
#@markdown - GMFSS.pkl : Animation
#@markdown - GMFSS_PRO.pkl : Animation
#@markdown - GIMMVFI_RAFT.pth : IRL
"""
INTERPOLATE_MODEL = "rife4.25.pkl"  #@param ["rife4.6.pkl", "rife4.7.pkl", "rife4.18.pkl", "rife4.22.pkl", "rife4.25.pkl", "GMFSS.pkl", "GMFSS_PRO.pkl", "GIMMVFI_RAFT.pth"]

#@markdown <hr>

#@markdown ##Upscaling settings
UPSCALE = False  #@param {type:"boolean"}

"""
#@markdown ##Select Upscale Models
#@markdown Upscale Models:
#@markdown - 4xNomos8k_span_otf_weak.pth : Realistic 4x High Quality Input
#@markdown - 4xNomos8k_span_otf_medium.pth : Realistic 4x Medium Quality Input
#@markdown - 4xNomos8k_span_otf_strong.pth : Realistic 4x Low Quality Input
#@markdown - 2x_BHI_SpanPlusDynamic_Light.pth : Realistic 2x High Quality Input (pytorch only, experimental)
#@markdown - 2x_ModernSpanimationV2.pth : Animation 2x
#@markdown - 2x_ModernSpanimationV3.pth : Animation 2x (pytorch only, experimental)
#@markdown - 2x_AnimeJaNai_V2_Compact_36k.pth : Animation 2x
#@markdown - 2x_AnimeJaNai_HD_V3_Sharp1_Compact_430k.pth : Animation 2x
#@markdown - up2x-latest-conservative.pth : Animation 2x (pytorch only, slow)
#@markdown - up3x-latest-conservative.pth : Animation 3x (pytorch only, slow)
#@markdown - up4x-latest-conservative.pth : Animation 4x (pytorch only, slow)
"""
UPSCALE_MODEL = "2x_ModernSpanimationV2.pth"  #@param ["4xNomos8k_span_otf_weak.pth", "4xNomos8k_span_otf_medium.pth", "4xNomos8k_span_otf_strong.pth", "2x_BHI_SpanPlusDynamic_Light.pth", "2x_ModernSpanimationV2.pth", "2x_ModernSpanimationV3.pth", "2x_AnimeJaNai_V2_Compact_36k.pth", "2x_AnimeJaNai_HD_V3_Sharp1_Compact_430k.pth", "up2x-latest-conservative.pth", "up3x-latest-conservative.pth", "up4x-latest-conservative.pth"]

#@markdown <hr>

#@markdown ##Encoder settings
VIDEO_ENCODER = "x264_nvenc"  #@param ["libx264", "libx265", "vp9", "av1", "prores", "ffv1", "x264_vulkan", "x264_nvenc", "x265_nvenc"]
AUDIO_ENCODER = "copy_audio"  #@param ["aac", "libmp3lame", "opus", "copy_audio"]
SUBTITLE_ENCODER = "copy_subtitle"  #@param ["srt", "ass", "webvtt", "copy_subtitle"]

#@markdown <hr>

#@markdown ##Backend choice
BACKEND = "pytorch"  #@param ["pytorch", "tensorrt"]


# 3. Install Requirement

In [ ]:
!rm -rf real-video-enhancer/
!git clone https://github.com/tntwise/real-video-enhancer --branch 2.2.1

In [ ]:
!mkdir real-video-enhancer/models/
origdir = os.getcwd()
os.chdir("real-video-enhancer/models/")


def has_model(model: str):
    return model and model.lower() != "none"

def download_model(model: str):
    os.system(f"wget https://github.com/TNTwise/real-video-enhancer-models/releases/download/models/{model}")

# get interpolate model by extracting the end of the link
def download_external_model(model_link : str):
        download_model(model_link)
        model = model_link.split(r'/')[-1]
        return model


if INTERPOLATE:
    if "https" in INTERPOLATE_MODEL:
        UPSCALE_MODEL = download_external_model(INTERPOLATE_MODEL, "interpolate")
    else:
        download_model(INTERPOLATE_MODEL)
if UPSCALE:
    if "https" in UPSCALE_MODEL:
        UPSCALE_MODEL = download_external_model(UPSCALE_MODEL, "upscale")
    else:
        download_model(UPSCALE_MODEL)


os.chdir(origdir)



In [ ]:
if BACKEND.lower() == 'tensorrt':
    !wget https://github.com/astral-sh/python-build-standalone/releases/download/20250317/cpython-3.12.9+20250317-x86_64-unknown-linux-gnu-install_only.tar.gz
    !tar xvf cpython-3.12.9+20250317-x86_64-unknown-linux-gnu-install_only.tar.gz
    !./python/bin/python3 -m pip install  --extra-index-url https://download.pytorch.org/whl/cu126 -r real-video-enhancer/backend/requirements.txt # only install if using tensorrt

!mkdir bin/ && cd bin && wget https://github.com/TNTwise/real-video-enhancer-models/releases/download/ffmpeg-colab/ffmpeg && chmod +x ffmpeg

# 4. Run

In [ ]:
def build_backend_command() -> str:
    command = (('python3 real-video-enhancer/backend/rve-backend.py' if BACKEND == "pytorch" else './python/bin/python3 real-video-enhancer/backend/rve-backend.py')
               + f' -i "{INPUT_FILE}"'
               + f' -o "{OUTPUT_FILE}"'
               + f' -b {BACKEND} '
               + f' --video_encoder_preset {VIDEO_ENCODER} '
               + f' --audio_encoder_preset {AUDIO_ENCODER} '
               + f' --subtitle_encoder_preset {SUBTITLE_ENCODER}')
    if UPSCALE_MODEL and UPSCALE:
        command += f" --upscale_model real-video-enhancer/models/{UPSCALE_MODEL} "
    if INTERPOLATE_MODEL and INTERPOLATE:
        command += f" --interpolate_model real-video-enhancer/models/{INTERPOLATE_MODEL} "
        command += f" --interpolate_factor {INTERPOLATE_FACTOR} "
        command += f" --scene_detect_method {SCENE_DETECT_METHOD} "
        command += f" --scene_detect_threshold {SCENE_DETECT_SENSATIVITY} "

    return command

# Write the command to a shell script
with open("command.sh", "w") as f:
    f.write(build_backend_command())

# Run the command from the shell script
!bash command.sh

# Remove the command.sh file after execution
import os
os.remove("command.sh")


# 5. Auto Kill

In [ ]:
# Source file path in Colab
source_path = f'/content/{OUTPUT_FILE}'

# Destination path on Google Drive (root directory)
destination_path = f'/content/drive/My Drive/RVE/{OUTPUT_FILE}'
if not os.path.exists(os.path.dirname(destination_path)):
  os.mkdir(os.path.dirname(destination_path))

# Move the file
try:
  shutil.copy(source_path, destination_path)
except Exception as e:
  print(f"Copying to drive failed! {e}\nPlease download any files that have correctly rendered before terminating the runtime!.")
  input("Press enter to terminate the runtime.")

  output.eval_js('google.colab.kernel.disconnect()')
  os.kill(os.getpid(), 9)

if KILL_RUNTIME_AFTER_RENDER:

  # kill the runtime
  output.eval_js('google.colab.kernel.disconnect()')
  os.kill(os.getpid(), 9)